In [52]:
import json
import time
import emoji
import pandas as pd
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import datasets
import re

In [53]:
def scroll_till_end(driver):
    height_now = driver.execute_script("return document.body.scrollHeight")
    height_end = height_now + 1
    while height_now != height_end:
        height_now = height_end
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        driver.implicitly_wait(0.2)
        height_end = driver.execute_script("return document.body.scrollHeight")
    print("end")

In [54]:
def get_urls(driver):
    try:
        find_links = driver.find_elements(By.CLASS_NAME, "tile-hover-target")
        products_urls = list(set([f'{link.get_attribute("href")}' for link in find_links]))
        print('[+] Ссылки были добавлены')
        return(products_urls)
    except:
        print("[!!!] Что-то пошло не так")

In [55]:
def get_list_of_reviews(driver,list_of_reviews):
    if len(driver.find_elements(By.CLASS_NAME, 'zq0_30')) > 0: # МЕНЯЙ ЭТО 
        
        reviews_list = driver.find_elements(By.CLASS_NAME, 'zq0_30')# МЕНЯЙ ЭТО
        
        for item in reviews_list:
            list_of_reviews.append(item.text.split('\n'))
        return(list_of_reviews)
        
    else:
        return(list_of_reviews)

In [56]:
# собирает отзывы со страницы и переходит на следующую страницу отзывов
def find_reviews(driver,list_of_reviews):
    driver.implicitly_wait(0.2)
    list_of_reviews = get_list_of_reviews(driver, list_of_reviews)
    if len(driver.find_elements(By.XPATH, '//*[@id="layoutPage"]/div[1]/div[2]/div/div[1]/div[7]/div/div[3]/div[2]/div[4]/div[3]/div/a[2]/div[2]')) > 0 :
        butt_next = driver.find_element(By.XPATH, '//*[@id="layoutPage"]/div[1]/div[2]/div/div[1]/div[7]/div/div[3]/div[2]/div[4]/div[3]/div/a[2]/div[2]')
        driver.implicitly_wait(0.2)
        
        butt_next.click()
        
        find_reviews(driver,list_of_reviews)
    return(list_of_reviews)

In [57]:
# собирает отзывы со страницы и переходит на следующую страницу отзывов для первого запуска
def find_reviews_first(driver,list_of_reviews):
    driver.implicitly_wait(2)
    list_of_reviews = get_list_of_reviews(driver, list_of_reviews)
    if len(driver.find_elements(By.XPATH, '//*[@id="layoutPage"]/div[1]/div[2]/div/div[1]/div[7]/div/div[3]/div[2]/div[4]/div[3]/div/a/div[2]')) > 0:
        butt_next = driver.find_element(By.XPATH, '//*[@id="layoutPage"]/div[1]/div[2]/div/div[1]/div[7]/div/div[3]/div[2]/div[4]/div[3]/div/a/div[2]')
        driver.implicitly_wait(2)
        
        butt_next.click()
        
        find_reviews(driver,list_of_reviews)
    return(list_of_reviews)

In [58]:
def convert_emoji(text):
    if emoji.emoji_count(text) > 0:
        return emoji.demojize(text)
    return text

In [59]:



def get_products_links(item_name = "PlayStation 5"):
    driver = uc.Chrome()
    driver.implicitly_wait(5)

    driver.get(url='https://ozon.ru')
    driver.implicitly_wait(1)

    find_input = driver.find_element(By.NAME, 'text')
    find_input.send_keys(item_name + '\n')

    scroll_till_end(driver)
    
    item_cards = driver.find_elements(By.CLASS_NAME,"s4w_30")

    data_test = []

    for item in item_cards:
        data_test.append(item.text)



    products_urls = get_urls(driver)
    products_urls_dicts = {}
    
    for k,v in enumerate(products_urls):
        products_urls_dicts.update({k: v})
    
    with open('products_urls_dict.json', 'w', encoding="utf-8") as file:
        json.dump(products_urls_dicts,file, indent = 4, ensure_ascii= False)
    



In [60]:
# функция для парсинга и разделения отзывов по категориям
def reviews_division(url):
    
    
    # подключаем undetected_chrome чтобы не банили на озоне 
    driver = uc.Chrome()
    
    # неявное ожидание делает то же самое что и time.sleep()
    driver.implicitly_wait(5)
    

    # создается массив для отзывов
    list_of_reviews = []
    print("HERE")
    # в зависимости от того какая ссылка передается она адаптируется, если требуется и добавляется
    # в конце reviews/ для открытия необходимой страницы
    if '?' in url:
        driver.get(url = url[0:url.find("?")] + 'reviews/')
    else:
        driver.get(url + 'reviews/')
    driver.implicitly_wait(5)
    # вызов функции для получения отзывов со страницы
    list_of_reviews = find_reviews_first(driver,list_of_reviews)
    driver.implicitly_wait(5)
    driver.close()
    return(list_of_reviews)

ЭТО ОСНОВА

In [61]:
def process_reviews_base(list_of_reviews):
    
    # создание массивов для хранения обработанных отзывов (без выделенных достоинств, недостатков и комментариев)
    # и для хранения отзывов с + - и комментариями 
    processed_lis_of_reviews = []
    list_to_post_process = []

    for item in list_of_reviews:
        # если в отзыве есть выделенные достоинства или недостатки или комментарии, то эти отзывы необходимо обработать далее
        if "Достоинства" in item or "Недостатки" in item or "Комментарий" in item:
            list_to_post_process.append(item[1:])
            
        # иначе этот отзыв уже обработан
        elif len(item) > 1:
            processed_lis_of_reviews.append(item[1:])

    # создаются массивы для плюсов минусов и комментариев
    pluses_list_of_reviews = []
    minuses_list_of_reviews = []
    commentary_list = []
    
    for item in list_to_post_process:

        if "Достоинства" in item:
            pluses_index = item.index("Достоинства") + 1
            pluses_list_of_reviews.append(item[pluses_index:pluses_index + 1])
        if "Недостатки" in item:
            minuses_index = item.index("Недостатки") + 1
            minuses_list_of_reviews.append(item[minuses_index:minuses_index + 1])
        if "Комментарий" in item:
            commentary_index = item.index("Комментарий") + 1
            commentary_list.append(item[commentary_index:commentary_index+1]) 
    #вывод для проверки 
    list_to_post_process__ = []
    pluses_list_of_reviews__ = []
    minuses_list_of_reviews__ = []
    commentary_list__ = []
    processed_lis_of_reviews__ = [] 
    pattern = r'\b\d{2}:\d{2}\b'
    print("--------------Для пост обработки----------------")
    for item in list_to_post_process:
        list_to_post_process__.append([re.sub(pattern,'',text.lower()) for text in item])
        print(item)
    print("-----------Достоинства-------------")
    for item in pluses_list_of_reviews:
        pluses_list_of_reviews__.append([re.sub(pattern,'',text.lower()) for text in item])
        print(item)
    print("-----------Недостатки-------------")
    for item in minuses_list_of_reviews:
        minuses_list_of_reviews__.append([re.sub(pattern,'',text.lower()) for text in item])
        print(item)
    print("-----------Комментарии-------------")
    for item in commentary_list:
        commentary_list__.append([re.sub(pattern,'',text.lower()) for text in item])
        print(item)
    print("-----------Отзывы без разделения-------------")
    for item in processed_lis_of_reviews:
        processed_lis_of_reviews__.append([re.sub(pattern,'',text.lower()) for text in item])
        print(item)
    print(list_to_post_process__,pluses_list_of_reviews__,minuses_list_of_reviews__,commentary_list__,processed_lis_of_reviews__)
    return pluses_list_of_reviews__, minuses_list_of_reviews__, commentary_list__, processed_lis_of_reviews__




ЭТО ПРОБНИК

In [62]:
def process_reviews_redactor(list_of_reviews):
    
    # создание массивов для хранения обработанных отзывов (без выделенных достоинств, недостатков и комментариев)
    # и для хранения отзывов с + - и комментариями 
    processed_lis_of_reviews = []
    list_to_post_process = []

    for item in list_of_reviews:
        # если в отзыве есть выделенные достоинства или недостатки или комментарии, то эти отзывы необходимо обработать далее
        if "Достоинства" in item or "Недостатки" in item or "Комментарий" in item:
            list_to_post_process.append(item[1:])
            print(item[1:], '---------------------------------------------s')
        # иначе этот отзыв уже обработан
        elif len(item) > 1:
                processed_lis_of_reviews.append(item[1:])

    # создаются массивы для плюсов минусов и комментариев
    pluses_list_of_reviews = []
    minuses_list_of_reviews = []
    commentary_list = []
    
    for item in list_to_post_process:

        if "Достоинства" in item:
            pluses_index = item.index("Достоинства") + 1
            pluses_list_of_reviews.append((item[pluses_index:pluses_index + 1]))
        if "Недостатки" in item:
            minuses_index = item.index("Недостатки") + 1
            minuses_list_of_reviews.append(item[minuses_index:minuses_index + 1])
        if "Комментарий" in item:
            commentary_index = item.index("Комментарий") + 1
            commentary_list.append(item[commentary_index:commentary_index+1])
    #вывод для проверки 
    list_to_post_process__ = []
    pluses_list_of_reviews__ = []
    minuses_list_of_reviews__ = []
    commentary_list__ = []
    processed_lis_of_reviews__ = [] 
    pattern = r'\b\d{2}:\d{2}\b'
    print("--------------Для пост обработки----------------")
    for item in list_to_post_process:
        text_without_smth = (re.sub(pattern,'',item[0].lower()))
        text_without_scobci = (re.sub(r"[\[\]\(\)\{\}]", "", text_without_smth))
        text_without_quotes = re.sub(r"[\"']", "", text_without_scobci)
        list_to_post_process__.append(text_without_quotes)
        print(item)
    print("-----------Достоинства-------------")
    for item in pluses_list_of_reviews:
        text_without_smth = (re.sub(pattern,'',item[0].lower()))
        text_without_scobci = (re.sub(r"[\[\]\(\)\{\}]", "", text_without_smth))
        text_without_quotes = re.sub(r"[\"']", "", text_without_scobci)
        pluses_list_of_reviews__.append(text_without_quotes)
        print(item)
    print("-----------Недостатки-------------")
    for item in minuses_list_of_reviews:
        text_without_smth = (re.sub(pattern,'',item[0].lower()))
        text_without_scobci = (re.sub(r"[\[\]\(\)\{\}]", "", text_without_smth))
        text_without_quotes = re.sub(r"[\"']", "", text_without_scobci)
        minuses_list_of_reviews__.append(text_without_quotes)
        print(item)
    print("-----------Комментарии-------------")
    for item in commentary_list:
        text_without_smth = (re.sub(pattern,'',item[0].lower()))
        text_without_scobci = (re.sub(r"[\[\]\(\)\{\}]", "", text_without_smth))
        text_without_quotes = re.sub(r"[\"']", "", text_without_scobci)
        commentary_list__.append(text_without_quotes)
        print(item)
    print("-----------Отзывы без разделения-------------")
    for item in processed_lis_of_reviews:
        text_without_smth = (re.sub(pattern,'',item[0].lower()))
        text_without_scobci = (re.sub(r"[\[\]\(\)\{\}]", "", text_without_smth))
        text_without_quotes = re.sub(r"[\"']", "", text_without_scobci)
        processed_lis_of_reviews__.append(text_without_quotes)
        print(item)
    print(list_to_post_process__,pluses_list_of_reviews__,minuses_list_of_reviews__,commentary_list__,processed_lis_of_reviews__)
    print("===========================================================")
    return pluses_list_of_reviews__, minuses_list_of_reviews__, commentary_list__, processed_lis_of_reviews__




In [63]:
# Функция для паддинга
def pad_array(arr, max_len, padding_value=None):
    return arr + [padding_value] * (max_len - len(arr))

In [64]:
# Создание датасета на основе данных
def create_dataset(pluses_list_of_reviews, minuses_list_of_reviews, commentary_list, processed_lis_of_reviews):

    # Найти максимальную длину массива
    max_len = max(len(pluses_list_of_reviews), len(minuses_list_of_reviews), len(commentary_list), len(processed_lis_of_reviews))

    # Применить паддинг
    pluses_list_of_reviews = pad_array(pluses_list_of_reviews, max_len, padding_value=None)
    minuses_list_of_reviews = pad_array(minuses_list_of_reviews, max_len, padding_value=None)
    commentary_list = pad_array(commentary_list, max_len, padding_value=None)
    processed_lis_of_reviews = pad_array(processed_lis_of_reviews, max_len, padding_value= None)
    df = pd.DataFrame({
        'Достоинства' : pluses_list_of_reviews,
        'Недостатки' : minuses_list_of_reviews,
        "Комментарии" : commentary_list,
        "Без разделения" : processed_lis_of_reviews
    })
    
    dataset_ = datasets.Dataset.from_pandas(df, preserve_index= False,)
    return df, dataset_
    

In [65]:
def save_dataset(df, name_of_dataset):
    df.to_csv(f'{name_of_dataset}', index = False,encoding='utf-8')

In [66]:
# Функция для дозаписи датасета
def fill_up_dataset(df_new, name_of_dataset):
    df_new.to_csv(f'{name_of_dataset}', mode = 'a', header = False, index = False)

In [67]:
def main():
    list_of_reviews = reviews_division("https://www.ozon.ru/product/igrovaya-konsol-playstation-5-slim-blu-ray-1316509567/?advert=ANsAzwJp7_gC8zdeLs3_IaKmfSL90imm7V_ABTSOSus_tKWRmJYqHJgG0rim_9fkmHnBjEhFYmYOaDau2WPhymQox__g6rj9oGm5vham4YDKL-vBJ_vRjKskLd5XHHfXpfy_5ThEBMae0j6i_3vxkxNYOdwad1-44tKU1rZ3uH3iHtUq_9ErCVtPZVFvLedhRcP00it08DZzOBPV2lUzv1BgfVhrraeKGWZ4XGQgg6IKgN4wYBBdpzxW2lZrCG-BMo9esQ03keGnJhDbHcmJom4qIlTL8oJqaZRCiBwpXHTnzZhuIdZFLwIVeuowBP9C7e7fQd_W9oQH2m591go8C7dQ196NoQrVKsl7ijUvL_1NdQwn_9zkrNw&avtc=1&avte=1&avts=1729784862&keywords=playstation+5")
    pluses_list_of_reviews, minuses_list_of_reviews, commentary_list, processed_lis_of_reviews = process_reviews_redactor(list_of_reviews)
    DATAFRAME_TO_WORK, DATASET_TO_WORK = create_dataset(pluses_list_of_reviews, minuses_list_of_reviews, commentary_list, processed_lis_of_reviews)
    save_dataset(DATAFRAME_TO_WORK,'parse_ozon_data.csv')
    print(DATASET_TO_WORK)

In [68]:

if __name__ == '__main__':
    main()

HERE
['Достоинства', 'Все работает '] ---------------------------------------------s
['Достоинства', 'Приставка рабочая, пришла быстро, купил по акции ОЗОН за 43т.р.!!! ', 'Недостатки', 'не было адаптера питания под китайскую вилку ', 'Комментарий', 'Версия CFI-2000 A1, была голубая транспортировочная наклейка, начальная заставка и игра AstroBoy в наличии, полная комплектация, упаковка целая, после обновления зашел в PSN (акк Турция), диск проверил, все работает отлично. '] ---------------------------------------------s
['Достоинства', 'оригинал, коробка в идеальном состоянии. за 43 огонь просто, чудом успел заказать когда появилась в наличии после 11 сентября '] ---------------------------------------------s
['Достоинства', 'Удалось купить по акции, повезло - коробка целая, без косяков, комплект полный: кроме консоли геймпад, три кабеля и ножки. Японская версия, 2000А, нужно докупать кабель или переходник на наши розетки. В работе еще не проверял, но надеюсь, что проблем не возникнет 